In [3]:
#On charge les modules dont on a besoin
using JuMP    #Pour la définition du modèle (variables, contraintes, fonction objectif, solveur à utiliser...)
using CPLEX   #Pour la résolution du modèle
using LinearAlgebra #Pour pouvoir utiliser des fonctions ou opérateurs de l'algèbre linéaire, comme la fonction

In [4]:
#Quelles sont les données du problème?

gravMin=[10,5,10]#gravier minimal pour les site de construction
gravMax=[10,18] #gravier max disponible
coutF1 = [130, 160, 150] #cout pour chaque site de F1
coutF2 = [180, 150, 160] #cout pour chaque site de F2
;

In [25]:
notreModeleNbsEntiers = Model(with_optimizer(CPLEX.Optimizer)) #on construit un modèle nommé notreModeleNbsEntiers
#et on indique que le solveur à utiliser est CPLEX

#On ajoute la variable x (un vecteur à 3 composants) au modèle notreModeleNbsEntiers.
@variable(notreModeleNbsEntiers, x[1:3] >=0, Int ) #Fournisseur 1
@variable(notreModeleNbsEntiers, y[1:3] >=0, Int ) #fournisseur 2
@variable(notreModeleNbsEntiers, v >=0, Int )#camion

@objective(notreModeleNbsEntiers, Min, dot(coutF1, x)+dot(coutF2,y) + 50*v)  #Fonction objectif

#Les contraintes
@constraint(notreModeleNbsEntiers, x[1]+y[1] >= gravMin[1])
@constraint(notreModeleNbsEntiers, x[2]+y[2] >= gravMin[2]) 
@constraint(notreModeleNbsEntiers, x[3]+y[3] >= gravMin[3]) 
@constraint(notreModeleNbsEntiers, x[1]+x[2]+x[3] <= gravMax[1]) 
@constraint(notreModeleNbsEntiers, y[1]+y[2]+y[3] <= gravMax[2]) 
@constraint(notreModeleNbsEntiers, v >= 5) 
@constraint(notreModeleNbsEntiers, (x[1]+x[2]+x[3]+y[1]+y[2]+y[3])/5 <= v) 

notreModeleNbsEntiers #Le modèle sera affiché

A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.Integer`: 7 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX
Names registered in the model: v, x, y

In [26]:
optimize!(notreModeleNbsEntiers) #On demande au solveur de résoudre le modèle

Tried aggregator 2 times.
MIP Presolve eliminated 1 rows and 0 columns.
MIP Presolve modified 1 coefficients.
Aggregator did 2 substitutions.
Reduced MIP has 4 rows, 5 columns, and 11 nonzeros.
Reduced MIP has 0 binaries, 5 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Found incumbent of value 3900.000000 after 0.00 sec. (0.03 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 3 rows and 4 columns.
MIP Presolve modified 3 coefficients.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.04 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.04 ticks)


In [27]:
termination_status(notreModeleNbsEntiers) #Pour savoir pourquoi l'optimisation a fini 

OPTIMAL::TerminationStatusCode = 1

In [28]:
#Pour les valeurs des variables
println("Solution: ", value.(x),value.(y), value.(v))
#Afficher la valeur optimale (la valeur de la fonction objectif)
println("Valeur optimale: ", objective_value(notreModeleNbsEntiers))

Solution: [10.0, 0.0, 0.0][0.0, 5.0, 10.0]5.0
Valeur optimale: 3900.0
